## 한글 폰트 설정

In [63]:
# 나눔바른고딕 설치, 설치된 폰트가 matplot에서 보여질 수 있도록 연동
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

%config InlineBackend.figure_format='retina'

!apt -qq -y install fonts-nanum

import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
plt.rc('font', family='NanumBarunGothic')
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


## 한국어 형태소 분석기 Mecab 설치
- 한국어 자연어 처리 konlpy 라이브러리
- 형태소 분석기 MeCab 설치
- 설치 명령어: !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [6]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-01-02 05:41:21--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-01-02 05:41:21--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca66ff69307d2444b961a2505fb.dl.dropboxusercontent.com/cd/0/get/BzymzL60Ri7DbAN5B-Zobmw7TtDPwHsN7ZJ5Czt6ywZdLgQeAoBi8k-Pi2EmRapJGkDpXWfR_y9i6DhIIYCJGL6hvriFoo_d0sF0cpwuuDPKruWL7bfKSgCRc39SQzwAKssTyMK0ugxM2h2uP4W4BJ4wwHquIvwygKRXxAxl9wZ-FFi4ciui4qr1jcNZtq6GRqQ/file?dl=1# [following]
--2023-01-02 05:41:21--  https://uca66ff69307d2444b961a2505fb.dl.dropboxusercontent.com/cd/0/get/BzymzL60Ri7

# 키워드 분석
* 핵심어(keyword)란 텍스트 자료의 중요한 내용을 압축적으로 제시하는 단어 또는 문구
* 핵심어 분석이란 불용어 제거와 어간추출 및 형태소 분석 등의 자연어 처리를 시행한 후 텍스트에서 많이 등장하는 형태소의 등장 빈도를 분석함으로써 핵심어를 
추출   
* 특정 텍스트 자료에 많이 나타나는 형태소가 그 텍스트 주제를 표출할 가능성이 높다는 가정에 기초  
* 물론 빈도 분석에서 영어의 전치사나 한국어의 조사와 같이 의미를 별로 담고 있지 않은 불용어는 제외하는 것이 좋음
* 키워드 분석은 텍스트의 주제 추정, 텍스트 유사도, 검색 엔진의 검색 결과 우선 순위 측정 등 다양하게 사용될 수 **있음**

In [8]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyLDAvis-3.3.1.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=ff5857ce1b5a67fdfe3749b95f275850d0ab803a0ad6cc127720a315f5f489ec
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=1607cebe21aa1f3450503aaa279fc014f6ffaac0ba2b951b02714cdd7ecbff92
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [9]:
import numpy as np
import pandas as pd
import warnings # 경고 메시지 무시
warnings.filterwarnings(action='ignore')
# 한국어 형태소 분석기 중 성능이 가장 우수한 Mecab 사용
from konlpy.tag import Mecab
mecab = Mecab()
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
import gensim # LDA 모델 활용 목적
import pyLDAvis.gensim_models # LDA 시각화용 패키지
from collections import Counter # 단어 등장 횟수 카운트

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


## 파일 불러오기

In [10]:
import pandas as pd

df = pd.read_excel('/content/윤석열대통령_연설문230101.xlsx')
df.head()

,순번,날짜,제목,내용
0,1,2022-05-22,취임사,"존경하고 사랑하는 국민 여러분, 750만 재외동포 여러분 그리고 자유를 사랑하는 세..."
1,2,2022-03-10,"정직한 정부, 정직한 대통령 되겠습니다.","위대하고 자랑스러운 국민 여러분! 고맙습니다. 다시 한 번, 진심으로 감사합니다.벅..."
2,3,2022-03-22,국민 속으로.. 용산에서 시작하겠습니다.​,국민 속으로.. 용산에서 시작하겠습니다.​ 대통령 집무실 이전은 새로운 시대의 출발...
3,4,2022-04-01,윤석열 정부의 대한민국은 청년이 주인공입니다.,청년들에게 무한한 기회를 제공하는 정부가 되겠습니다. 단기적인 지원이 아닌 목표에 ...
4,5,2022-04-01,국민이 안심할 수 있는 사회,국가를 위해 희생한 분들이 분노하지 않는 나라를 만들겠다고 분명히 약속 드렸습니다....


In [11]:
# 데이터 탐색

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순번      85 non-null     int64         
 1   날짜      85 non-null     datetime64[ns]
 2   제목      85 non-null     object        
 3   내용      85 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.8+ KB


In [12]:
def clean_text(d):
  pattern = r'[^a-zA-Z\s]'
  text = re.sub(pattern,'',d)
  return text

In [13]:
news_df = df['내용']
news_df.head()

0    존경하고 사랑하는 국민 여러분, 750만 재외동포 여러분 그리고 자유를 사랑하는 세...
1    위대하고 자랑스러운 국민 여러분! 고맙습니다. 다시 한 번, 진심으로 감사합니다.벅...
2    국민 속으로.. 용산에서 시작하겠습니다.​ 대통령 집무실 이전은 새로운 시대의 출발...
3    청년들에게 무한한 기회를 제공하는 정부가 되겠습니다. 단기적인 지원이 아닌 목표에 ...
4    국가를 위해 희생한 분들이 분노하지 않는 나라를 만들겠다고 분명히 약속 드렸습니다....
Name: 내용, dtype: object

In [14]:
# drop_na 하기
news_df.replace("", float("Nan"), inplace = True)
news_df.dropna(inplace= True)
print(len(news_df))

85


In [15]:
news_df.head()

0    존경하고 사랑하는 국민 여러분, 750만 재외동포 여러분 그리고 자유를 사랑하는 세...
1    위대하고 자랑스러운 국민 여러분! 고맙습니다. 다시 한 번, 진심으로 감사합니다.벅...
2    국민 속으로.. 용산에서 시작하겠습니다.​ 대통령 집무실 이전은 새로운 시대의 출발...
3    청년들에게 무한한 기회를 제공하는 정부가 되겠습니다. 단기적인 지원이 아닌 목표에 ...
4    국가를 위해 희생한 분들이 분노하지 않는 나라를 만들겠다고 분명히 약속 드렸습니다....
Name: 내용, dtype: object

In [57]:
# 전처리용 dictionary 불러오기 

stopword_list = pd.read_excel('/content/stopword_list.xlsx')
stopword_list.head()

,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


In [58]:
# 한글자인 키워드 리스트를 불러옵니다. 

one_char_keyword = pd.read_excel('/content/one_char_list.xlsx')
one_char_keyword.head()

,one_char_keyword
0,컵
1,방
2,물
3,돈
4,꿈


In [59]:
content_tokenized = list(map(lambda review: mecab.nouns(review), news_df))

In [60]:
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list['stopword']):
                review_removed_stopword.append(token)
        # 토큰의 글자 수가 1글자인 경우
        else:
            # 1글자 키워드에 포함되는 경우만 분석용 리뷰 데이터로 포함
            if token in list(one_char_keyword['one_char_keyword']):
                review_removed_stopword.append(token)
    return review_removed_stopword

In [61]:
content_removed_stopword = list(map(lambda tokens : remove_stopword(tokens), content_tokenized))

In [62]:
content_removed_stopword

[['존경',
  '사랑',
  '국민',
  '재외',
  '동포',
  '자유',
  '사랑',
  '세계',
  '시민',
  '나라',
  '자유민주주의',
  '시장',
  '경제',
  '체제',
  '기반',
  '국민',
  '주인',
  '나라',
  '재건',
  '국제',
  '사회',
  '책임',
  '역할',
  '나라',
  '시대',
  '소명',
  '자리',
  '역사',
  '자리',
  '국민',
  '감사',
  '문재인',
  '박근혜',
  '대통령',
  '리마',
  '야콥',
  '싱가포르',
  '대통령',
  '포스',
  '아르',
  '투아',
  '중앙아프리카',
  '공화국',
  '대통령',
  '왕치산',
  '중국',
  '국가',
  '부주석',
  '메가와티',
  '수카르노푸트리',
  '인도네시아',
  '대통령',
  '더글러스',
  '호프',
  '해리스',
  '미국',
  '부통령',
  '부군',
  '조지',
  '퓨리',
  '캐나다',
  '상원',
  '의장',
  '하야시',
  '요시마사',
  '일본',
  '외무',
  '세계',
  '각국',
  '경축',
  '사절',
  '내외',
  '귀빈',
  '감사',
  '자리',
  '코로나',
  '펜데',
  '극복',
  '과정',
  '고통',
  '감내',
  '국민',
  '경의',
  '헌신',
  '의료진',
  '감사',
  '존경',
  '국민',
  '세계',
  '시민',
  '세계',
  '팬데',
  '위기',
  '교역',
  '질서',
  '변화',
  '공급',
  '재편',
  '기후',
  '변화',
  '식량',
  '에너지',
  '위기',
  '분쟁',
  '평화',
  '해결',
  '후퇴',
  '나라',
  '독자',
  '나라',
  '참여',
  '해결',
  '난제',
  '직면',
  '위기',
  '복합',
  '인류',
  '사회',
  '그림자',
  '나라',

In [63]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(content_removed_stopword[i])
    detokenized_doc.append(t)

df['headline_text'] = detokenized_doc # 다시 text['headline_text']에 재저장

In [64]:
df['headline_text'][:5]

0    존경 사랑 국민 재외 동포 자유 사랑 세계 시민 나라 자유민주주의 시장 경제 체제 ...
1    자랑 국민 감사 마음 무한 책임감 대통령 당선인 국민 공직 사퇴 국민 지지 성원 정...
2    국민 용산 시작 대통령 집무실 시대 출발점 소통 대통령실 구현 부처 군림 권력 독점...
3    청년 무한 기회 제공 정부 단기 지원 목표 안착 실질 도움 정부 최선 목소리 청년 ...
4               국가 희생 분노 나라 약속 윤석열 국가 희생 대한민국 국민 영웅 기억
Name: headline_text, dtype: object

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features= 100) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(df['headline_text'])
X.shape # TF-IDF 행렬의 크기 확인

(85, 100)

In [75]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=7,learning_method='online',max_iter=1)    #random_state=777,

In [76]:
lda_top=lda_model.fit_transform(X)

In [77]:
print(lda_model.components_)
print(lda_model.components_.shape)

[[0.83304419 0.90707701 1.00637236 0.83989224 0.9252768  0.74698668
  0.84240016 0.94460986 0.75827634 0.792801   0.88122425 0.94225718
  0.91803724 0.78947554 1.13486169 0.88245727 1.15371433 0.91918692
  0.80771224 0.87082729 0.83260828 1.02275109 0.79129228 0.83343715
  1.05755812 0.89126233 1.04482457 0.84062141 1.02895926 0.85771239
  0.96530997 0.79007795 0.84152017 0.9616107  0.98335835 0.78674362
  0.82769659 0.8925583  0.94220768 0.93238204 0.90605989 0.92571741
  0.85034269 0.76580216 0.9766715  1.04744879 0.7811702  0.79181011
  1.04500349 0.90112399 0.96988025 0.96450435 0.8049439  0.91980063
  1.04209479 1.16497302 0.8366505  0.82327638 0.9569473  1.02232067
  0.85390876 0.96407946 0.88428871 0.71415628 0.76455265 0.96586089
  1.14024159 0.95949277 0.97513991 1.01119828 0.92059565 0.83164764
  1.0779914  0.89975287 0.95686838 0.96931476 0.94632836 0.77351456
  0.8860064  0.85031363 0.8175873  0.70690928 0.91286903 1.0418738
  0.77362465 0.80824378 0.8664449  0.80142027 0.7

In [78]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('시장', 1.16), ('국가', 1.15), ('원전', 1.14), ('교육', 1.13), ('적극', 1.08), ('기술', 1.06), ('부처', 1.05), ('사회', 1.05), ('나라', 1.04), ('수출', 1.04)]
Topic 2: [('국민', 1.33), ('나라', 1.28), ('정부', 1.24), ('변화', 1.19), ('자리', 1.18), ('미래', 1.17), ('안정', 1.16), ('위기', 1.15), ('지원', 1.13), ('산업', 1.13)]
Topic 3: [('국민', 1.41), ('자유', 1.27), ('대응', 1.16), ('피해', 1.16), ('신속', 1.14), ('안보', 1.13), ('관계', 1.12), ('개선', 1.11), ('당부', 1.11), ('재정', 1.1)]
Topic 4: [('국제', 1.33), ('역할', 1.29), ('양국', 1.28), ('대응', 1.19), ('금융', 1.17), ('위기', 1.16), ('사회', 1.15), ('경제', 1.15), ('대한민국', 1.13), ('필요', 1.13)]
Topic 5: [('국민', 2.01), ('피해', 1.25), ('국가', 1.23), ('안전', 1.13), ('시장', 1.13), ('개혁', 1.13), ('사회', 1.11), ('약자', 1.1), ('디지털', 1.1), ('정부', 1.1)]
Topic 6: [('역할', 1.13), ('중요', 1.1), ('신속', 1.1), ('협력', 1.1), ('대통령', 1.09), ('공공', 1.08), ('안전', 1.05), ('필요', 1.05), ('정상', 1.05), ('사회', 1.04)]
Topic 7: [('경제', 1.29), ('과학', 1.18), ('산업', 1.18), ('국민', 1.15), ('지원', 1.15), ('양국', 1.14), ('규제', 1.